In [2]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget -q $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget -q $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install -q ta-lib
!pip install -q valinvest
!python -m textblob.download_corpora
import talib

(Reading database ... 160732 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk

In [3]:
!cp drive/MyDrive/mySecrets2.py .
import mySecrets2
#API Keys
apiKey = mySecrets2.stockSecrets['fmpApiKey']
valinvestKey = mySecrets2.stockSecrets['valinvestKey']



In [4]:
from textblob import TextBlob
import pandas as pd
import datetime as dt
import requests
import numpy as np
import valinvest
from scipy import stats
from scipy.stats import linregress
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipywidgets as widgets
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot

In [5]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 75 # 200 e.g. is really fine, but slower
pd.set_option('display.max_colwidth', None)

In [6]:
# Todo : Return on Assets > 10% 
# The average company in the S&P 500 Index has a ROA around 2.7%.
# I like seeing a company that has a ROA of at least 10%.
# I like seeing a company that has a net debt-to-EBITDA of less than 4.
# I like seeing a company that has a price-to-earnings multiple of less than 25.
# I like seeing a company that has multiple years of consecutive revenue growth.

In [7]:
# Magic Formula
#  Magic formula investing only applies to companies that have market caps of more than $100 million. 
# The strategy also excludes financial companies, utility companies, and non-U.S. companies.
# After determining their earnings yield (EBIT / enterprise value) and return on capital (EBIT / (net fixed assets + working capital), 
# companies are ranked by highest earnings yield and highest return on capital. Usually, investors are recommended to invest in the top 20-30 
# companies from this ranked list, accumulating 2-3 positions per month over a period of one year. This process is repeated over a period of 5-10 years. 
# Portfolios are re-balanced by ejecting losers or underperformers one week before the year-term ends and selling winners one week after the year mark.

In [8]:
def getSentimentScore(text):
  """The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective."""
  blob = TextBlob(text)
  return(blob.sentiment.polarity)


In [9]:
class Stock:
  
  def __init__(self, ticker):
    self.ticker = ticker
    self.price = None
    self.volAvg = None
    self.industry = None
    self.sector = None 
    self.mktCap = None
    self.changesPercentage = None
    self.change = None
    self.dayLow = None
    self.dayHigh = None
    self.yearHigh = None 
    self.yearLow = None
    self.priceAvg50 = None
    self.priceAvg200 = None
    self.open = None
    self.previousClose = None
    self.eps = None
    self.pe = None
    self.sectorPerformance = None
    self.growthDf = None
    self.DCF = None
    self.grahamNumber = None
    self.earningsSurpriseDelta = None
    self.dfEarningsSurprises = None
    self.dfAnnualKeyMetrics = None
    self.companyRatingsRating = None
    self.ratingRecommendation = None
    self.ratingDetailsDCFRecommendation = None
    self.ratingDetailsROARecommendation = None
    self.ratingDetailsDERecommendation = None
    self.ratingDetailsROERecommendation = None
    self.ratingDetailsPERecommendation = None
    self.ratingDetailsPBRecommendation = None
    self.valinvestFScore = None
    self.sentiment = None
  def printCompanyStats(self):
    print(f"Ticker : {self.ticker}")
    print(f"Sector : {self.sector}")
    print(f"Market Cap : {self.mktCap}")
    print(f"EPS : {self.eps}")
    print(f"P/E Ratio : {self.pe}")
    print(f"Daily High : {self.dayHigh}")
    print(f"Daily Low : {self.dayLow}")
    print(f"50 Day Avg : {self.priceAvg50}")
    print(f"200 Day Avg : {self.priceAvg200}")
    print(f"Previous Day Close : {self.previousClose}")
    print(f"Year High : {self.yearHigh}")
    print(f"Year Low : {self.yearLow}")
    print(f"Price Change : {self.change}")
    print(f"Price % Change : {self.changesPercentage}")
    print(f"Sector Performce  : {self.sectorPerformance}")
  def getCompanyProfile(self):
    url = "https://financialmodelingprep.com/api/v3/profile/"+ self.ticker + apiKey
    # print(f"getCompanyProfile URL : {url}")
    df = pd.read_json(url)
    self.price = df.iloc[0]['price']
    self.volAvg = df.iloc[0]['volAvg']
    self.industry = df.iloc[0]['industry']
    self.sector = df.iloc[0]['sector'] 
    self.mktCap = df.iloc[0]['mktCap']
  def getQuote(self):
    url = "https://financialmodelingprep.com/api/v3/quote/"+ self.ticker + apiKey
    # print(f"getQuote : {url}")
    df = pd.read_json(url)
    self.changesPercentage = df.iloc[0]['changesPercentage']
    self.change = df.iloc[0]['change']
    self.dayLow = df.iloc[0]['dayLow']
    self.dayHigh = df.iloc[0]['dayHigh']
    self.yearHigh = df.iloc[0]['yearHigh'] 
    self.yearLow = df.iloc[0]['yearLow']
    self.priceAvg50 = df.iloc[0]['priceAvg50']
    self.priceAvg200 = df.iloc[0]['priceAvg200']
    self.open = df.iloc[0]['open']
    self.previousClose = df.iloc[0]['previousClose']
    self.eps = df.iloc[0]['eps']
    self.pe = df.iloc[0]['pe']
    self.price = df.iloc[0]['price']
  def getSectorPerformace(self):
    url = "https://financialmodelingprep.com/api/v3/stock/sectors-performance" + apiKey
    response = requests.get(url)
    df = response.json()['sectorPerformance']
    df2 = pd.json_normalize(df)
    self.sectorPerformance = df2.loc[df2['sector'] == self.sector, "changesPercentage"].values[0]
  def get_finvizNews(self):
    try:
      url = ("http://finviz.com/quote.ashx?t=" + self.ticker.lower())
      req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
      webpage = urlopen(req).read()
      html = soup(webpage, "html.parser")
      # Find news table
      news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
      links = []
      for a in html.find_all('a', class_="tab-link-news"):
          links.append(a['href'])
      # Clean up news dataframe
      news.columns = ['Date', 'News Headline']
      news['Article Link'] = links
      news = news.set_index('Date')
      print(f"News: {news}")
      return news

    except Exception as e:
      return e
  def getGrowthStats(self):
    url = "https://financialmodelingprep.com/api/v3/income-statement-growth/" + self.ticker + apiKey
    df = pd.read_json(url,convert_dates=True)
    self.growthDf = df
  def getHistoricalStockPrices(self):
    url = "https://financialmodelingprep.com/api/v3/historical-price-full/" + self.ticker + apiKey
    response = requests.get(url)
    response = response.json()
    final = pd.json_normalize(response["historical"])
    self.stockPricesHistory = final
  def getAnnualKeyMetrics(self):
    url = "https://financialmodelingprep.com/api/v3/key-metrics/" + self.ticker + apiKey
    df = pd.read_json(url,convert_dates=True)
    df.sort_values(by='date', inplace=True, ascending=True)
    grahamNumber = df['grahamNumber'].iloc[-1]
    self.dfAnnualKeyMetrics = df
    print(f"grahamNumber : {grahamNumber}")
    self.grahamNumber = grahamNumber
    display(df)
  def getEarningsSurprises(self):
    url = "https://financialmodelingprep.com/api/v3/earnings-surpises/" + self.ticker + apiKey
    df = pd.read_json(url,convert_dates=True)
    df["Actual vs Estimate"] = df["actualEarningResult"] - df["estimatedEarning"]
    df["Actual Rating"] = df.apply(lambda row: "Beat" if row["Actual vs Estimate"] >0  else 'Lose', axis = 1)
    # df.apply(lambda row:  print(f'Row TEST : {row["Actual vs Estimate"]}' ), axis = 1)
    display(df)
    self.dfEarningsSurprises = df
    self.earningsSurpriseDelta = df["actualEarningResult"].values[0] / df["estimatedEarning"].values[0] *100
  def getCompanyRatings(self):
    url = "https://financialmodelingprep.com/api/v3/rating/" + self.ticker + apiKey
    df = pd.read_json(url,orient='records')
    self.companyRatingsDF = df
    self.companyRatingsRating = df["rating"].values[0]
    self.ratingRecommendation = df["ratingRecommendation"].values[0]
    self.ratingDetailsDCFRecommendation = df["ratingDetailsDCFRecommendation"].values[0]
    self.ratingDetailsROARecommendation = df["ratingDetailsROARecommendation"].values[0]
    self.ratingDetailsDERecommendation = df["ratingDetailsDERecommendation"].values[0]
    self.ratingDetailsROERecommendation = df["ratingDetailsROERecommendation"].values[0]
    self.ratingDetailsPERecommendation = df["ratingDetailsPERecommendation"].values[0]
    self.ratingDetailsPBRecommendation = df["ratingDetailsPBRecommendation"].values[0]
  def getFScoreValinvest(self):
    fScore = valinvest.Fundamental(self.ticker, valinvestKey)
    self.valinvestFScore  = fScore.fscore()  
    print(f"Valinvest F-Score for {self.ticker} : {self.valinvestFScore}")
  def getFmpNews(self):
    url = "https://financialmodelingprep.com/api/v3/stock_news?tickers="+ self.ticker + "&limit=150&" + apiKey.strip("?")
    df = pd.read_json(url)
    # convert the 'Date' column to datetime format
    df['publishedDate']= pd.to_datetime(df['publishedDate'])
    df['SentimentScore'] = df.apply(lambda row: getSentimentScore(row["text"]), axis = 1)
    overallScore = round(df.SentimentScore.mean(),4)
    range_max = df['publishedDate'].max()
    range_min = range_max - dt.timedelta(days=3)
    sliced_df = df[(df['publishedDate'] >= range_min) & 
               (df['publishedDate'] <= range_max)]
    score = round(sliced_df.SentimentScore.mean(), 4)
    if(score >= .5):
      sentimentScore = "Very Positive"
    elif(score >=0 and  score < .5):
      sentimentScore = "Positive"
    elif(score >=-.5 and score < 0):
      sentimentScore = "Negative"
    else:
      sentimentScore = "Very Negative"
    print(f"Current Sentiment for {self.ticker} is : {sentimentScore} : {score} compared to longer term value of {overallScore}")
    self.sentiment = sentimentScore


 
  

In [10]:
# Input
# ticker = input('Enter a ticker: ')
ticker = "AAPL"

In [11]:
stock = Stock(ticker)
stock.getCompanyProfile()
stock.getQuote()
stock.getSectorPerformace()
stock.get_finvizNews()
stock.getGrowthStats()
stock.getHistoricalStockPrices()
stock.stockPricesHistory['SMA(5)'] = stock.stockPricesHistory.adjClose.rolling(5).mean()
stock.stockPricesHistory['SMA(20)'] = stock.stockPricesHistory.adjClose.rolling(20).mean()
stock.stockPricesHistory['RSI'] = talib.RSI(stock.stockPricesHistory.adjClose, timeperiod=14)
stock.stockPricesHistory['StochRSIIndicator'] = talib.RSI(stock.stockPricesHistory.adjClose, timeperiod=14)
stock.stockPricesHistory['ROC'] = talib.ROC(stock.stockPricesHistory.adjClose, 10)
stock.stockPricesHistory['KAMA'] = talib.KAMA(stock.stockPricesHistory.adjClose,timeperiod=15)
display(stock.stockPricesHistory)
stock.getEarningsSurprises()
stock.getAnnualKeyMetrics()
stock.printCompanyStats()
stock.getCompanyRatings()
stock.getFScoreValinvest()
stock.getFmpNews()

News:                                                                                                                                                                          News Headline                                                                                                                                                                              Article Link
Date                                                                                                                                                                                                                                                                                                                                                                            
May-25-21 02:19PM                                                                                                Epic-Apple Judge Signals Her Ruling Wont Please Either Side Bloomberg                                                                          

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,SMA(5),SMA(20),RSI,StochRSIIndicator,ROC,KAMA
0,2021-05-24,126.010000,127.940000,125.9400,127.100000,127.100000,61196681.0,61196681.0,1.0900,0.865,126.99333,"May 24, 21",0.00865,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-21,127.820000,128.000000,125.2100,125.430000,125.430000,79388743.0,79388743.0,-2.3900,-1.870,126.21333,"May 21, 21",-0.01870,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-05-20,125.230000,127.720000,125.1000,127.310000,127.310000,73359943.0,73359943.0,2.0800,1.661,126.71000,"May 20, 21",0.01661,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-05-19,123.160000,124.920000,122.8800,124.690000,124.690000,86428018.0,86428018.0,1.5300,1.242,124.16333,"May 19, 21",0.01242,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-05-18,126.560000,126.990000,124.7800,124.850000,124.850000,60196256.0,60196256.0,-1.7100,-1.351,125.54000,"May 18, 21",-0.01351,125.876000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,2016-06-01,24.754999,24.885000,24.5825,24.615000,23.068445,116693200.0,116693200.0,-0.1400,-0.566,24.69417,"June 01, 16",-0.00566,23.043137,22.675884,52.894483,52.894483,1.358876,22.672551
1254,2016-05-31,24.900000,25.100000,24.7050,24.965000,23.396452,169228800.0,169228800.0,0.0650,0.261,24.92333,"May 31, 16",0.00261,23.082030,22.749335,58.863728,58.863728,2.462557,22.732357
1255,2016-05-27,24.860001,25.117500,24.8125,25.087500,23.511253,145364800.0,145364800.0,0.2275,0.915,25.00583,"May 27, 16",0.00915,23.162628,22.846683,60.738967,60.738967,3.092247,22.838963
1256,2016-05-26,24.920000,25.182501,24.6600,25.102501,23.525312,225324800.0,225324800.0,0.1825,0.732,24.98167,"May 26, 16",0.00732,23.279306,22.928803,60.973597,60.973597,1.598695,22.930402


,date,symbol,actualEarningResult,estimatedEarning,Actual vs Estimate,Actual Rating
0,2021-04-28,AAPL,1.40,0.99,0.41,Beat
1,2021-01-27,AAPL,1.68,1.41,0.27,Beat
2,2020-10-29,AAPL,0.73,0.70,0.03,Beat
3,2020-07-30,AAPL,0.65,0.51,0.14,Beat
4,2020-04-30,AAPL,0.64,0.56,0.08,Beat
5,2020-01-28,AAPL,1.25,1.14,0.11,Beat
6,2019-10-30,AAPL,0.76,0.71,0.05,Beat
7,2019-07-30,AAPL,0.55,0.53,0.02,Beat
8,2019-04-30,AAPL,0.62,0.59,0.03,Beat
9,2019-01-29,AAPL,1.05,1.04,0.01,Beat


grahamNumber : 16.74258602902124


,symbol,date,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
40,AAPL,1985-09-27,FY,0.138344,0.004400,0.019114,0.015219,0.024308,0.039671,0.067513,0.039671,0.000000,1.114069e+09,7.770694e+08,18.263434,0.580850,4.204036,5.279950,2.025581,2.025581,0.405146,4.796725,2.932338,3.682794,0.054754,0.189396,0.701818,0.412393,-2.080247,2.786441,NaN,4.344262,NaN,0.000000,0.000000,0.038060,0.000000,-4.907407,-35.518519,-0.777778,0.000000,0.062668,0.187207,0.065171,0.014390,527000000,NaN,436000000,NaN,23012000000,23155500000,2136500000,41.866528,24.485689,54.521467,8.718182,14.906667,6.694611,0.110909,-0.003895
39,AAPL,1986-09-26,FY,0.132010,0.010689,0.021863,0.017213,0.039978,0.048168,0.080511,0.048168,0.000000,2.122601e+09,1.546601e+09,13.783123,1.115984,6.738416,8.558875,3.058503,3.058503,0.813145,4.284213,4.909844,6.236294,0.072552,0.116838,0.671470,0.401724,-1.595568,3.164134,NaN,2.045455,NaN,0.000000,0.000000,0.067298,0.000000,-4.701493,-28.388060,-0.761194,0.000000,0.107629,0.373045,0.132759,0.025108,712000000,NaN,575000000,NaN,23033500000,23177000000,2107500000,50.470557,48.338945,44.652076,7.231939,7.550847,8.174312,0.221902,-0.004650
38,AAPL,1987-09-25,FY,0.180529,0.014722,0.012754,0.006920,0.038331,0.056716,0.100271,0.056716,0.000000,4.672049e+09,4.107049e+09,21.530181,1.755749,24.851326,45.804405,5.588576,5.588576,1.543423,8.538564,21.846007,40.265190,0.046446,0.021832,0.766746,0.433694,-1.174636,2.728601,NaN,0.866359,0.003211,0.069124,0.000000,0.072153,0.000000,-2.186047,-30.941860,-0.825581,0.000000,0.137065,0.410410,0.146820,0.023168,828000000,NaN,666000000,NaN,23105000000,23221000000,2166000000,55.689590,58.016975,63.649691,6.554187,6.291262,5.734513,0.259569,-0.005834
37,AAPL,1988-09-30,FY,0.279832,0.027495,0.020209,0.010311,0.025571,0.068944,0.143112,0.068944,0.021652,5.000875e+09,4.943875e+09,12.502187,1.228414,17.009779,33.339167,4.985917,4.985917,1.214413,6.735525,16.815901,32.959167,0.079986,0.029995,1.075773,0.518252,-0.077657,2.155985,NaN,0.735000,0.007999,0.100000,0.000000,0.067060,0.000000,-2.041667,-28.270833,-0.541667,0.000000,0.206523,0.522709,0.192123,0.022151,956000000,NaN,704000000,NaN,23132500000,23159500000,2097500000,41.332596,15.215972,16.315922,8.830803,23.987952,22.370787,0.398804,-0.009898
36,AAPL,1989-09-29,FY,0.366639,0.031502,0.035179,0.018596,0.030391,0.103109,0.190397,0.103109,0.003955,5.822707e+09,5.441707e+09,12.825347,1.101951,11.484630,21.726520,3.918376,3.918376,1.029846,6.262034,10.733151,20.304878,0.077971,0.046027,0.846568,0.458455,-0.438435,2.563128,NaN,1.116740,0.008587,0.110132,0.000000,0.079485,0.000000,-2.121339,-22.108787,-0.523013,0.000000,0.270336,0.402380,0.165452,0.026592,1399000000,NaN,1036000000,NaN,23298500000,23285000000,2290500000,54.777631,45.235622,64.332096,6.663304,8.068862,5.673684,0.305518,-0.016583
35,AAPL,1989-09-30,FY,0.367868,0.031607,0.035318,0.018679,0.030514,0.103433,0.103433,0.103433,0.003954,5.867371e+09,5.485871e+09,12.923724,1.110403,11.565880,21.868695,3.949230,3.949230,1.038204,7.226809,10.813859,20.446779,0.077377

Ticker : AAPL
Sector : Technology
Market Cap : 2109813161984
EPS : 4.449
P/E Ratio : 28.418745
Daily High : 128.3168
Daily Low : 126.41
50 Day Avg : 129.91972
200 Day Avg : 127.3064
Previous Day Close : 127.1
Year High : 145.09
Year Low : 78.2725
Price Change : -0.665
Price % Change : -0.52
Sector Performce  : -0.0678%
Valinvest F-Score for AAPL : 6.9
Current Sentiment for AAPL is : Positive : 0.0656 compared to longer term value of 0.0768


In [12]:
def getSectorPEratio(sector):
  url = "https://www.gurufocus.com/sector_shiller_pe.php"
  try:
    dfs = pd.read_html(url)
    df = pd.concat(dfs)
    df = df[df['Sector'].str.match(sector)]
    sectorPE = df.iloc[0]["Regular P/E"]
  except:
    sectorPE ="N/A"
    print(f'Could not find sector : {sector} in the getSectorPEratio function')
  return(sectorPE)

In [13]:

class StockScreenerCriteria():
  def __init__(self, stock):
    self.percentOfHighRating = None
    self.earningsSurpriseRating = None
    self.stock = stock
    self.buyThreshold = 85
    self.earningsSurpriseThreshold = 105
    self.momentumScore = None 
    self.roeRating = None
    self.roe5yearAvg = None
    self.DCF = None
    self.DCFRating = None
    self.grahamNumberRating = None
    self.valinvestFScoreRating = None
   
  def getPercentof52weekHigh(self):
    print(f"Price {self.stock.price}")
    print(f"Year High {self.stock.yearHigh}")
    # Stocks that are near or approaching their 52-week highs tend to be reinforced by 
    # either already reported strong earnings or anticipated strong earnings. 
    # The chart is a reflection of a company's performance. And when a stock is positioned 
    # to set a new high, it is a very positive indicator.
    self.percentOfHigh = (self.stock.price / self.stock.yearHigh) * 100
    #set buy or sell tag based on result and store
    # print(f"52 week high : {self.stock.yearHigh}")
    # print(f"Current Price : {self.stock.price}")
    print(f"Current price as a percent of the 52 week high : {self.percentOfHigh}")
    if (self.percentOfHigh >= self.buyThreshold):
      self.percentOfHighRating = "Buy"
      print(f"Rating : {self.percentOfHighRating}")
    else:
      self.percentOfHighRating = "Sell"
      print(f"Rating : {self.percentOfHighRating}")

  def getMomentumScore(self):
    x = np.arange(len(self.stock.stockPricesHistory ["adjClose"]))
    log_ts = np.log(self.stock.stockPricesHistory ["adjClose"])
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    anualized_slope = (np.power(np.exp(slope), 252)-1 )*100
    score = anualized_slope * (r_value **2)
    self.momentumScore = score 
    print(f"Momentum Score = {score}")

  def getValueROE(self): 
    historicalROE = self.stock.dfAnnualKeyMetrics["roe"]
    roeRange = range(len(historicalROE))
    slope, intercept, rvalue, pvalue, stderr =linregress(roeRange, historicalROE)
    if slope > 0 : 
      self.roeRating = "Buy"
    else:
      self.roeRating = "Sell"
    if self.stock.dfAnnualKeyMetrics['roe'].mean() > .2 : 
      self.roe5yearAvg = "Buy"
    else:
      self.roe5yearAvg = "Sell"
    print(f"The ROE has a slope of {slope} giving it a rating : {self.roeRating} with a 5 year average of {self.stock.dfAnnualKeyMetrics['roe'].mean()} {self.roe5yearAvg}")

  def getEarningsSurprise(self):
    print(f"Delta of Estimated Earnings vs Actual Earnings : {self.stock.earningsSurpriseDelta}")
    if (self.stock.earningsSurpriseDelta >= self.earningsSurpriseThreshold):
      self.earningsSurpriseRating = "Buy"
      print(f"Rating : {self.earningsSurpriseRating}")
    else:
      self.earningsSurpriseRating = "Sell"
      print(f"Rating : {self.earningsSurpriseRating}") 
    self.stock.dfEarningsSurprises.sort_values(by='date', inplace=True, ascending=True)
    xAxisRange = range(len(self.stock.dfEarningsSurprises["actualEarningResult"]))
    actualRange = self.stock.dfEarningsSurprises["actualEarningResult"]
    slope, intercept, rvalue, pvalue, stderr =linregress(xAxisRange, actualRange)
    print(f"The last 4 quarters actual earnings has a slope of {slope} giving it a rating : {self.earningsSurpriseRating} ")
  
  def peRatioVSIndustryPE(self):
    print(f"Type : {type(self.stock.pe)}")
    if type(self.stock.pe) != int or float:
      self.peVsSectorAvgRating = "N/A"
      return(self.peVsSectorAvgRating)
    print(f"{self.stock.ticker} current PE Ratio {self.stock.pe}")
    print(f"{self.stock.ticker} belongs to the  {self.stock.sector} sector")
    sectorAvgPE = getSectorPEratio(self.stock.sector)
    if(self.stock.pe > sectorAvgPE):
      #stock over priced
      self.peVsSectorAvgRating = "Sell"
    else:
      #stock a value compared to sector
      self.peVsSectorAvgRating = "Buy"
    print(f"{self.stock.sector} Avg PE Ratio {sectorAvgPE}")
    return(self.peVsSectorAvgRating)
  def getDiscountedCashFlow(self):
    print("Function getDiscountedCashFlow")
    # if DCF is lower than the current stock price the stock is overvalued -> SELL
    url = "https://financialmodelingprep.com/api/v3/discounted-cash-flow/"+ self.stock.ticker + apiKey
    # print(url)
    df = pd.read_json(url)
    self.DCF = df["dcf"].values[0]
    if self.DCF == "inf":
      self.DCF = -1000000
    print(f"DCF = {self.DCF}")
    if (self.DCF < self.stock.price):
      # if DCF is lower than the current stock price the stock is overvalued -> SELL
      self.DCFRating = "Sell"
    else: 
      self.DCFRating = "Buy"
    print(f"DCF is {self.DCF} compared to current stock price {self.stock.price} yielding a rating of : {self.DCFRating}")
    return(self.DCFRating)

  def getGrahamsScore(self):
    # What Is the Graham Number? The Graham number is a figure that measures a stock's fundamental value by taking into account the company's earnings per share and book value per share. 
    # The Graham number is the upper bound of the price range that a defensive investor should pay for the stock. According to the theory, any stock price below the Graham number is considered 
    # undervalued and thus worth investing in.
    if(self.stock.grahamNumber < self.stock.price):
      #if graham number > stock price its over priced and SELL
      self.grahamNumberRating = "Sell"
    else:
      self.grahamNumberRating = "Buy"
    
    print(f"The Graham number is {self.stock.grahamNumber} vs the current stock price of {self.stock.price} yeilding a rating of {self.grahamNumberRating}")

  def getCalculatedRatings(self):
    for column in self.stock.companyRatingsDF:
      value = self.stock.companyRatingsDF[column].values[0]
      # print(f"Value: {type(value)}")
      if( type(value) == str ):
        print(f"Rating: {column} -> {value}")

  def getModifiedPiotroskiF_Score(self):
    # https://seekingalpha.com/article/4407684-why-piotroskis-f-score-no-longer-works
    fScore = 0
    # Positive three-year average retained earnings
    # Sum of TTM cash flow from operations and cash flow from investments greater than 10% of revenue
    # At least eight of last twelve quarters’ EPS greater than same quarter previous year
    # Cash flow from operations greater than net income each of last three fiscal years
    # TTM EBITDA greater than one-third of total debt
    # Current ratio greater than one
    # TTM equity purchased greater than equity issued
    # TTM gross margin greater than subsector median
    # Average five-year asset turnover greater than subsector median

  def getFScoreValinvestRating(self):
    if(self.stock.valinvestFScore < 7):
      #if graham number > stock price its over priced and SELL
      self.valinvestFScoreRating = "Sell"
    else:
      self.valinvestFScoreRating = "Buy"
    print(f"The Valinvest F-Score number is {self.stock.valinvestFScore}  yeilding a rating of {self.valinvestFScoreRating}")




In [14]:
screener = StockScreenerCriteria(stock)
screener.getPercentof52weekHigh()
screener.getEarningsSurprise()
screener.getMomentumScore()
screener.getValueROE()
screener.peRatioVSIndustryPE()
screener.getDiscountedCashFlow()
screener.getGrahamsScore()
screener.getCalculatedRatings()
screener.getFScoreValinvestRating()

Price 126.435
Year High 145.09
Current price as a percent of the 52 week high : 87.14246329864223
Rating : Buy
Delta of Estimated Earnings vs Actual Earnings : 141.41414141414143
Rating : Buy
The last 4 quarters actual earnings has a slope of 0.057090909090909095 giving it a rating : Buy 
Momentum Score = -25.416386790943367
The ROE has a slope of 0.00724817129083399 giving it a rating : Buy with a 5 year average of 0.21265759773781337 Buy
Type : <class 'numpy.float64'>
Function getDiscountedCashFlow
DCF = 128.85884618136222
DCF is 128.85884618136222 compared to current stock price 126.435 yielding a rating of : Buy
The Graham number is 16.74258602902124 vs the current stock price of 126.435 yeilding a rating of Sell
Rating: symbol -> AAPL
Rating: rating -> S
Rating: ratingRecommendation -> Strong Buy
Rating: ratingDetailsDCFRecommendation -> Strong Buy
Rating: ratingDetailsROERecommendation -> Strong Buy
Rating: ratingDetailsROARecommendation -> Neutral
Rating: ratingDetailsDERecommen